In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{game}_upcache', connect_args={'connect_timeout': 10})

def get_latency_sql(game,min_time,time_threshold,tier_or_spend_threshold = ''):
    return f"""
           select {game}_metrics.time AS day, {game}_metrics.user_id,
           attacker_upcache.country_tier AS attacker_tier,
           defender_upcache.country_tier AS defender_tier,
           ROUND(IFNULL(defender_upcache.money_spent,0),0) AS defender_spend,
           max(1.0*spec) AS latency,
           count(distinct({game}_metrics.time)) AS N
           from {game}_metrics
           inner join {game}_upcache attacker_upcache on attacker_upcache.user_id = {game}_metrics.user_id
           inner join {game}_upcache defender_upcache on defender_upcache.user_id = {game}_metrics.stack
           where time >= {min_time} and event_name = '3973_on_approach_latency'
           and {game}_metrics.stack != {game}_metrics.user_id
           and (1.0*spec) >= {time_threshold}
           group by {game}_metrics.time, {game}_metrics.user_id, stack
           {tier_or_spend_threshold}
           """

def get_pve_latency_sql(game,min_time):
    return f"""
            select FROM_UNIXTIME({game}_metrics.time) AS day,
            {game}_metrics.user_id,
            max(1.0*spec) AS latency
            from {game}_metrics
            where time >= {min_time} and event_name = '3973_on_approach_latency'
            and {game}_metrics.stack != {game}_metrics.user_id
            and {game}_metrics.stack < 1112
            and (1.0*spec) >= 30.0
            group by {game}_metrics.time, {game}_metrics.user_id, stack"""

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sat, 22 Jan 2022 at 21:54:26 UTC


In [12]:
# Incidents vs. Tier 1/2 and superfan defenders

min_time = time.time() - (14 * 86400)
min_time_str = datetime.date.fromtimestamp(time.time()) - datetime.timedelta(14)
updated_time = min_time_str.strftime('%a, %d %b %Y')
for game in ('tr','dv','fs','bfm','mf2'):
    engine = get_engine(game)
    latency_sql = get_latency_sql(game,min_time,6,"having (defender_tier in ('1','2') or defender_spend > 10)")
    latency_data = pd.read_sql(latency_sql, engine)
    if len(latency_data) > 0:
        print('')
        gamename = game.upper().replace('MF2','WSE')
        print(f'Latency Incidents vs. Tier 1/2 and superfan defenders ({gamename}), showing incidents since {updated_time}')
        header_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format('Date', 'User ID', 'Attacker Tier', 'Defender Tier', 'Defender Spend', 'Latency')
        print(header_line)
        for i, day in enumerate(latency_data['day']):
            formatted_date = datetime.date.fromtimestamp(day).strftime('%Y-%m-%d')
            user_id = latency_data['user_id'][i]
            attacker_tier = latency_data['attacker_tier'][i]
            defender_tier = latency_data['defender_tier'][i]
            defender_spend = latency_data['defender_spend'][i]
            latency = latency_data['latency'][i]
            output_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format(formatted_date, str(user_id), str(attacker_tier),str(defender_tier),str(defender_spend), str(latency))
            print(output_line)
        else:
            print(f'Checked Latency Incidents vs. Tier 1/2 and superfan defenders ({gamename}), since {updated_time}, but none found.')


Latency Incidents vs. Tier 1/2 and superfan defenders (TR), showing since Sat, 08 Jan 2022 at 00:00:00 UTC
Date         User ID    Attacker Tier  Defender Tier  Defender Spend  Latency 
2022-01-09   800226     4              3              121.0           8.1     
2022-01-09   800226     4              3              121.0           7.1     
2022-01-09   4196291    4              1              0.0             10.62   
2022-01-09   4196291    4              2              0.0             7.83    
2022-01-10   4196291    4              2              0.0             8.16    
2022-01-10   4196291    4              4              17.0            7.56    
2022-01-10   4196291    4              4              17.0            13.38   
2022-01-10   2914135    4              4              350.0           7.51    
2022-01-11   4196291    4              2              332.0           21.79   
2022-01-11   4196291    4              2              332.0           31.79   
2022-01-12   1130682   

In [14]:
# prepare data for Incidents vs. Tier 1/2 and superfan defenders

min_time = time.time() - (14 * 86400)
min_time_str = datetime.date.fromtimestamp(time.time()) - datetime.timedelta(14)
updated_time = min_time_str.strftime('%a, %d %b %Y')
for game in ('tr','dv','fs','bfm','mf2'):
    engine = get_engine(game)
    latency_sql = get_latency_sql(game,min_time,20,"having N >= 4")
    latency_data = pd.read_sql(latency_sql, engine)
    if len(latency_data) > 0:
        print('')
        gamename = game.upper().replace('MF2','WSE')
        print(f'Latency Incidents vs. All defenders (20sec+, N >= 4) ({gamename}), showing since {updated_time}')
        header_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format('Date', 'User ID', 'Attacker Tier', 'Defender Tier', 'Defender Spend', 'Latency')
        print(header_line)
        for i, day in enumerate(latency_data['day']):
            formatted_date = datetime.date.fromtimestamp(day).strftime('%Y-%m-%d')
            user_id = latency_data['user_id'][i]
            attacker_tier = latency_data['attacker_tier'][i]
            defender_tier = latency_data['defender_tier'][i]
            defender_spend = latency_data['defender_spend'][i]
            latency = latency_data['latency'][i]
            output_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format(formatted_date, str(user_id), str(attacker_tier),str(defender_tier),str(defender_spend), str(latency))
            print(output_line)
    else:
        print(f'Checked Latency Incidents vs. All defenders (20sec+, N >= 4) ({gamename}), since {updated_time}, but none found.')

In [ ]:
# prepare data for Incidents in PvE

min_time = time.time() - (14 * 86400)
min_time_str = datetime.date.fromtimestamp(time.time()) - datetime.timedelta(14)
updated_time = min_time_str.strftime('%a, %d %b %Y')
for game in ('tr','dv','fs','bfm','mf2'):
    engine = get_engine(game)
    latency_sql = get_pve_latency_sql(game,min_time)
    latency_data = pd.read_sql(latency_sql, engine)
    if len(latency_data) > 0:
        print('')
        gamename = game.upper().replace('MF2','WSE')
        print(f'Latency Incidents in PvE (30sec+) ({gamename}), showing since {updated_time}')
        header_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format('Date', 'User ID', 'Attacker Tier', 'Defender Tier', 'Defender Spend', 'Latency')
        print(header_line)
        for i, day in enumerate(latency_data['day']):
            formatted_date = datetime.date.fromtimestamp(day).strftime('%Y-%m-%d')
            user_id = latency_data['user_id'][i]
            attacker_tier = latency_data['attacker_tier'][i]
            defender_tier = latency_data['defender_tier'][i]
            defender_spend = latency_data['defender_spend'][i]
            latency = latency_data['latency'][i]
            output_line = "{0:12} {1:10} {2:14} {3:14} {4:15} {5:8}".format(formatted_date, str(user_id), str(attacker_tier),str(defender_tier),str(defender_spend), str(latency))
            print(output_line)
    else:
        print(f'Checked Latency Incidents in PvE (30sec+) ({gamename}) since {updated_time}, but none found.')